In [1]:
import os

from folioclient import FolioClient

okapi_url = os.getenv("OKAPI_URL")
user = os.getenv("OKAPI_USER")
password = os.getenv("OKAPI_PASSWORD")
tenant = os.getenv("TENANT")

folio_client = FolioClient(
    okapi_url,
    tenant,
    user,
    password)

In [4]:
folio_funds = folio_client.folio_get(
         "/finance-storage/funds", key="funds", query_params={"limit": 2999}
     )
fund = folio_funds[0]
fund.pop("externalAccountNo", None)
print(fund)

{'id': '8527a67d-f099-4a19-b104-7cd8969569b1', '_version': 5, 'allocatedFromIds': [], 'allocatedToIds': [], 'code': 'ASULRES-SUL', 'description': 'ALT FUND: RESMAT', 'fundStatus': 'Active', 'fundTypeId': 'e228d3cd-c0bc-4089-89eb-0ce100278327', 'ledgerId': 'a0d6c701-c316-48d4-bac9-76a34103a3c9', 'name': 'ASULRES', 'acqUnitIds': ['bd6c5f05-9ab3-41f7-8361-1c1e847196d3'], 'tags': {'tagList': ['sulacq:funds:moderate_restricted']}, 'metadata': {'createdDate': '2023-08-14T17:22:11.326+00:00', 'createdByUserId': '58d0aaf6-dcda-4d5e-92da-012e6b7dd766', 'updatedDate': '2024-07-03T06:22:05.115+00:00', 'updatedByUserId': 'b296db77-f6c8-44fb-b16d-2281a9f37b01'}}


## Path 1: Use payment transactions to start
**FOLIO UI uses this query:**
```
query="?query=((fiscalYearId=200bfabe-07c7-4deb-b54e-99d64a3435cb and (fromFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411 or toFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411)) and transactionType==\"Payment\") sortby transactionDate/sort.descending"
```

We don't need to limit by fiscal year.

In [5]:
query="?query=((fiscalYearId=200bfabe-07c7-4deb-b54e-99d64a3435cb and (fromFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411 or toFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411)) and transactionType==\"Payment\") sortby transactionDate/sort.descending"
ui_response = folio_client.folio_get("/finance/transactions", query=query)
print(ui_response['totalRecords'])

20


**Get all payment transactions for a digital bookplate fund**

transactionType=Payment

fund ID = UUID of fund

In [6]:
query = {'query':'transactionType=Payment and (fromFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411 or toFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411)'}
total_transactions = folio_client.folio_get("/finance/transactions", query_params=query, key="totalRecords")
print(total_transactions)

21


**Using folio_get_all function to get all transactions**

folio_get_all cannot take a dict for the query. You must pass a string for query.

In [7]:
query = "?query=transactionType==Payment and (fromFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411 or toFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411)"
transactions = folio_client.folio_get_all("/finance/transactions", key="transactions", query=query, limit=999)

**Get the invoice line ID from the transaction**

In [8]:
invoice_lines = []
for x in transactions:
    # print(x)
    invoice_lines.append(x['sourceInvoiceLineId'])

print(len(invoice_lines))
# print(invoice_lines)

20


**Get the PO line ID from the invoice line**

Not all invoice lines will have a po line. For instance, if the invoice line is for SHIPPING.

In [9]:
transactions_query_po_lines = []
for x in invoice_lines:
    invoice_line = folio_client.folio_get(f"/invoice/invoice-lines/{x}")
    #print(invoice_line)
    po_line_id = invoice_line.get('poLineId')
    if po_line_id is not None:
        transactions_query_po_lines.append(po_line_id)

print(len(transactions_query_po_lines))
# print(transactions_query_po_lines)

18


Maybe we don't need to start at transactions because we can query invoice lines on paid invoices with fund distributions for each of the digital bookplate funds.

# Start with invoice UUID paid within date range

In [10]:
from_date = "2023-08-28"
to_date = "2023-09-01"
query = f"""?query=((paymentDate>={from_date} and paymentDate<={to_date}) and status=="Paid")"""
invoices = folio_client.folio_get_all("/invoice/invoices", key="invoices", query=query, limit=500)
# print([row.get("id") for row in invoices])
invoice_ids = []
for row in invoices:
    invoice_ids.append(row["id"])

print(invoice_ids)

['9cf2899a-c7a6-4101-bf8e-c5996ded5fd1', '02e89573-8dda-46ad-8142-4ad9f37463f5', '255cb8af-74b6-4943-87dc-30d81efb81b1']


**Get invoice lines from invoice uuids**

In [11]:
all_invoice_lines = []
for id in invoice_ids:
    print(f"Getting invoice lines for {id}")
    query = f"""?query=(invoiceId=={id})"""
    invoice_lines = folio_client.folio_get_all("/invoice/invoice-lines", key="invoiceLines", query=query, limit=500)
    # print([row for row in invoice_lines])
    for row in invoice_lines:
        all_invoice_lines.append(row)

# print(all_invoice_lines)

invoice_lines_funds_polines = []
for row in all_invoice_lines:
    fund_poline = {
        "fund_ids": None,
        "poline_id": None
    }
    fund_ids = []
    fund_ids = [x["fundId"] for x in row.get("fundDistributions")]
    fund_poline["fund_ids"] = fund_ids
    poline_id = row.get("poLineId")
    if poline_id is not None:
        fund_poline["poline_id"] = poline_id

    lines_funds_polines = {}
    lines_funds_polines[row["id"]] = fund_poline
    invoice_lines_funds_polines.append(lines_funds_polines)
    
print(invoice_lines_funds_polines)
        

Getting invoice lines for 9cf2899a-c7a6-4101-bf8e-c5996ded5fd1
Getting invoice lines for 02e89573-8dda-46ad-8142-4ad9f37463f5
Getting invoice lines for 255cb8af-74b6-4943-87dc-30d81efb81b1
[{'fadacf66-8813-4759-b4d3-7d506db38f48': {'fund_ids': ['0e8804ca-0190-4a98-a88d-83ae77a0f8e3'], 'poline_id': 'b5ba6538-7e04-4be3-8a0e-c68306c355a2'}}, {'a16030c1-66ca-44c1-b0a3-572cde626685': {'fund_ids': ['47e1fc24-300d-4817-a866-5c0a2f490522'], 'poline_id': '5513c3d7-7c6b-45ea-a875-09798b368873'}}, {'99340beb-9bf5-401c-8f99-91ff19464087': {'fund_ids': ['47e1fc24-300d-4817-a866-5c0a2f490522'], 'poline_id': None}}, {'e2363b0c-1a82-40b2-8601-efa5d8ebc859': {'fund_ids': ['47e1fc24-300d-4817-a866-5c0a2f490522'], 'poline_id': '0e6796b4-498a-45e7-bf2d-696f79c77a5d'}}]


## Path 2: Use invoice lines to start

In [12]:
query = "?query=fundDistributions==\"*3eb86c5f-c77b-4cc9-8f29-7de7ce313411*\" and invoiceLineStatus==\"Paid\""
invoice_lines = folio_client.folio_get_all("/invoice/invoice-lines", key="invoiceLines", query=query, limit=999)

In [13]:
invoices_query_po_lines = []
for x in invoice_lines:
    invoice_line_id = x['id']
    invoice_line = folio_client.folio_get(f"/invoice/invoice-lines/{invoice_line_id}")
    # print(invoice_line)
    po_line_id = invoice_line.get('poLineId')
    if po_line_id is not None:
        invoices_query_po_lines.append(po_line_id)

print(len(invoices_query_po_lines))
print(invoices_query_po_lines)
    

20
['13e05f54-343e-4daf-acf9-3ae6fc66f5e1', '7a5888fe-689b-4cfe-a27d-c2675a235203', 'be0af62c-665e-4178-ae13-e3250d89bcc6', 'e8cef55e-2bdc-4c1c-980d-3c66e493798b', 'c18b9e40-ad39-40dd-b918-1bd2b999fdaf', '798596da-12a6-4c6d-8d3a-3bb6c54cb2f1', '5dc4cfb3-a86f-4cfe-86f9-949fbe84aa17', '2fb3c6ce-4541-4c19-8e16-d8ed6dd7a9f9', '9047a74c-1057-4ddb-b52d-d01429b0b450', 'd55342ce-0a33-4aa2-87c6-5ad6e1a12b75', 'c18b9e40-ad39-40dd-b918-1bd2b999fdaf', 'e399f1e9-6d10-4116-98d4-e7d192a00e57', '13e05f54-343e-4daf-acf9-3ae6fc66f5e1', '1d45bbb7-63e1-47e9-9b45-75a06f2c7590', 'e8cef55e-2bdc-4c1c-980d-3c66e493798b', '87319dc7-a5ea-4380-9b99-b02ec15638ed', '39bbf04f-e729-4681-b805-bff3e4b933ac', 'c3159ebb-7d49-4eee-933a-7a63636e4750', '24491205-2f0c-4034-bdbb-f715d0410780', '08ad0d3a-a906-4bf7-8dc5-28cf3964daae']


There really should not be a difference in the number of po lines we get from querying the transactions endpoint versus the invoice lines endpoint. If there is, then it is likely the query is not being passed correctly to okapi.

In [14]:
# print("PO Lines from paid transactions via invoice lines:", sorted(transactions_query_po_lines))
# print("PO Lines from paid invoices' invoice lines:", sorted(invoices_query_po_lines))
def diff(first, second):
        second = set(second)
        return [item for item in first if item not in second]

additional_po_lines_from_transactions = diff(transactions_query_po_lines, invoices_query_po_lines)
print(additional_po_lines_from_transactions)
additional_po_lines_from_invoice_lines = diff(invoices_query_po_lines, transactions_query_po_lines)
print(additional_po_lines_from_invoice_lines)

[]
[]


## Get paid invoices in paymentDate range

In [15]:
from_date = "2023-08-28T00:00:00+00:00"
query = f"""?query=((paymentDate>={from_date}) and status=="Paid")"""
# print(query)
invoices = folio_client.folio_get_all("/invoice/invoices", key="invoices", query=query, limit=999)
# for row in invoices:
#     print(row['id'])
# print([row.get("id") for row in invoices])



## Get the instance ID from PO lines
Some of the po lines don't have an instance ID because they are for a package. Example: https://folio.stanford.edu/orders/lines/view/9047a74c-1057-4ddb-b52d-d01429b0b450?limit=50&offset=0&qindex=poLineNumber&query=55487-1

Outstanding question about what to do with these (report out?).

In [16]:
instances = []
for x in invoices_query_po_lines:
    order_line = folio_client.folio_get(f"/orders/order-lines/{x}")
    instance_id = order_line.get('instanceId')
    if instance_id is None:
        print(f"PO Line {order_line['poLineNumber']} not linked to a folio Instance record.")
    else:
        print(instance_id)
        instances.append(instance_id)



06660d4f-982d-54e8-b34c-532c268868e1
0a97a746-87ce-58f1-bb61-ffe47b39aff8
a5a1b279-1a00-5af2-9bbb-b5657fa15078
772fe534-028f-5579-b2b0-7079f404cd88
1cb88f99-5c59-55e8-a834-e45a6a95f778
242c6000-8485-5fcd-9b5e-adb60788ca59
98f6a7fa-21af-5b08-834b-3f7e97457834
0991463f-5be0-544b-a827-ffc628594a50
PO Line 55487-1 not linked to a folio Instance record.
0410d399-b34d-5830-85d5-ce815dbb3c5d
1cb88f99-5c59-55e8-a834-e45a6a95f778
947cd292-a543-5ba7-b5b2-59f9e2523b1e
06660d4f-982d-54e8-b34c-532c268868e1
7a465bf3-ab73-4c30-bd7b-28a7905a5b3e
772fe534-028f-5579-b2b0-7079f404cd88
857421b8-5704-43b2-99fd-3a5f285b3f9e
2816efec-41a1-4d79-803a-95769f33be09
7327db79-7f3c-42e1-9614-afe9ed1ca2a3
e361da4f-5fbe-53d8-9fdc-7bb6ab378490
8fcdaf3b-808e-4fde-8fa4-d3ad58363c7f


**Get the MARC for each instance to do a PUT to add 979**

Outstanding question: will the 979 be added to the marc-bib mapping for updating the instance json?

We probably could refactor the [OCLCAPIWrapper class](https://github.com/sul-dlss/libsys-airflow/blob/d6b6f1c2b7b0db4b231520edd92ef814bccd3932/libsys_airflow/plugins/data_exports/oclc_api.py#L75) to do a GET to change-manager/parsedRecords since it will give us the data we need to do a PUT (SRS record ID, instance hrid). There is a relatedRecordVersion field from change-manager/parsedRecords that should be the instance version (for optimistic locking) but I think it is a new field and we do not have it populated in our data at that endpoint. It is a required field so we need to get it from inventory/instances.

In [18]:
for instance_id in instances:
    instance_version = folio_client.folio_get(f"/inventory/instances/{instance_id}").get('_version')
    query = {'externalId': f"{instance_id}"}
    parsed_record = folio_client.folio_get("/change-manager/parsedRecords", query_params=query)
    record_fields = parsed_record['parsedRecord']['content']['fields']
    record_979s = []
    field_979 = next((field for field in record_fields if field.get('979')), None)
    record_979s.append(field_979)
    print(f"id: {parsed_record['id']}", f"instance id: {parsed_record['externalIdsHolder']['instanceId']}", f"instance hrid: {parsed_record['externalIdsHolder']['instanceHrid']}", f"record state: {parsed_record['recordState']}", f"version: {parsed_record.get('relatedRecordVersion', instance_version)}", f"979 fields: {record_979s}", sep="\n")

id: e60b77d3-3a76-59e2-88f7-3d1a045af3b1
instance id: 06660d4f-982d-54e8-b34c-532c268868e1
instance hrid: a477973
record state: ACTUAL
version: 3
979 fields: [{'979': {'ind1': ' ', 'ind2': ' ', 'subfields': [{'f': 'ABBOTT'}, {'b': 'druid:ws066yy0421'}, {'c': 'ws066yy0421_00_0001.jp2'}, {'d': 'The The Donald P. Abbott Fund for Marine Invertebrates'}]}}]
id: e51ece5e-d620-5227-babf-c32024751f6b
instance id: 0a97a746-87ce-58f1-bb61-ffe47b39aff8
instance hrid: a358123
record state: ACTUAL
version: 3
979 fields: [{'979': {'ind1': ' ', 'ind2': ' ', 'subfields': [{'f': 'ABBOTT'}, {'b': 'druid:ws066yy0421'}, {'c': 'ws066yy0421_00_0001.jp2'}, {'d': 'The The Donald P. Abbott Fund for Marine Invertebrates'}]}}]
id: c6aaa60d-bea2-5787-9c0a-ff6bf1c256ce
instance id: a5a1b279-1a00-5af2-9bbb-b5657fa15078
instance hrid: a366033
record state: ACTUAL
version: 91
979 fields: [{'979': {'ind1': ' ', 'ind2': ' ', 'subfields': [{'f': 'ABBOTT'}, {'b': 'druid:ws066yy0421'}, {'c': 'ws066yy0421_00_0001.jp2'}, {'

RemoteProtocolError: Server disconnected without sending a response.